In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import beta       
import seaborn as sns
import random
from sklearn.model_selection import train_test_split

Getting dataset from the files

In [41]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='latin1')
        # X=datadict['data'].reshape((len(datadict['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
        # dict_new=dict['data'].reshape((len(dict['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
    return dict

def NormalizeData(X):
    X = X.astype('float64')
    X /= np.std(X, axis = 0)
batch_1 = unpickle("data_batch_1")
batch_2 = unpickle("data_batch_2")
batch_3 = unpickle("data_batch_3")
batch_4 = unpickle("data_batch_4")
batch_5 = unpickle("data_batch_5")
# batch_test = unpickle("test_batch")
# batch_meta = unpickle("batches.meta")
features_1 = np.array(batch_1['data'].reshape((len(batch_1['data']), 3, 32, 32)).transpose(0, 2, 3, 1))
features_2 = np.array(batch_2['data'].reshape((len(batch_2['data']), 3, 32, 32)).transpose(0, 2, 3, 1))
features_3 = np.array(batch_3['data'].reshape((len(batch_3['data']), 3, 32, 32)).transpose(0, 2, 3, 1))
features_4 = np.array(batch_4['data'].reshape((len(batch_4['data']), 3, 32, 32)).transpose(0, 2, 3, 1))
features_5 = np.array(batch_5['data'].reshape((len(batch_5['data']), 3, 32, 32)).transpose(0, 2, 3, 1))
NormalizeData(features_1[:])
NormalizeData(features_2[:])
NormalizeData(features_3[:])
NormalizeData(features_4[:])
NormalizeData(features_5[:])



Multilayer perceptron algorithm:

In [42]:
logistic = lambda z: 1./ (1 + np.exp(-z))

class MLP:
    
    def __init__(self, M = 64):
        self.M = M
            
    def fit(self, x, y, optimizer):
        N,D = x.shape
        def gradient(x, y, params):
            v, w = params
            z = logistic(np.dot(x, v)) #N x M
            yh = logistic(np.dot(z, w))#N
            dy = yh - y #N
            dw = np.dot(z.T, dy)/N #M
            dz = np.outer(dy, w) #N x M
            dv = np.dot(x.T, dz * z * (1 - z))/N #D x M
            dparams = [dv, dw]
            return dparams
        
        w = np.random.randn(self.M) * .01
        v = np.random.randn(D,self.M) * .01
        params0 = [v,w]
        self.params = optimizer.run(gradient, x, y, params0)
        return self
    
    def predict(self, x):
        v, w = self.params
        z = logistic(np.dot(x, v)) #N x M
        yh = logistic(np.dot(z, w))#N
        return yh

In [43]:
class SGD:
    
    def __init__(self, learning_rate=.001, max_iters=1e4, epsilon=1e-8, epochs = 100, batch_size = 1):
        self.learning_rate = learning_rate
        self.max_iters = max_iters
        self.epsilon = epsilon
        self.epochs = epochs
        self.batch_size = batch_size
        
    def run(self, gradient_fn, x, y, params):
        norms = np.array([np.inf])
        t = 1
        total_samples = x.shape[0]
        while np.any(norms > self.epsilon) and t < self.max_iters:
            tup = (x,y)
            random.suffle(tup)
            mini_batches = [
                tup[z: z+self.batch_size]
                for z in range(0,total_samples,self.batch_size)
            ]
            for batch in mini_batches:
                gd(batch)
            grad = gradient_fn(x, y, params)
            for p in range(len(params)):
                params[p] -= self.learning_rate * grad[p]
            t += 1
            norms = np.array([np.linalg.norm(g) for g in grad])
        return params